In [39]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [40]:
df_ramen = pd.read_excel('The-Big-List-20210117.xlsx')

In [75]:
df_ramen.head()

,Review #,Brand,Variety,Style,Country,Stars,T
0,3702,Higashimaru,Seafood Sara Udon,Pack,Japan,5,NaN
1,3701,Single Grain,Chongqing Spicy & Sour Rice Noodles,Cup,China,3.5,NaN
2,3700,Sau Tao,Seafood Flavour Sichuan Spicy Noodle,Pack,Hong Kong,5,NaN
3,3699,Sau Tao,Jiangnan Style Noodle - Original Flavour,Pack,Hong Kong,4.5,NaN
4,3698,Sapporo Ichiban,CupStar Shio Ramen,Cup,Japan,3.5,NaN


In [42]:
df_ramen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3702 entries, 0 to 3701
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Review #  3702 non-null   int64  
 1   Brand     3702 non-null   object 
 2   Variety   3702 non-null   object 
 3   Style     3702 non-null   object 
 4   Country   3702 non-null   object 
 5   Stars     3702 non-null   object 
 6   T         0 non-null      float64
dtypes: float64(1), int64(1), object(5)
memory usage: 202.6+ KB


In [43]:
# Remove leading and trailing spaces in each string value
for col in df_ramen[['Brand', 'Variety','Style','Country']]:
    df_ramen[col] = df_ramen[col].str.strip()
    print('Number of Unique Values in ' + str(col) + ': ' + str(df_ramen[col].nunique()))

Number of Unique Values in Brand: 543
Number of Unique Values in Variety: 3448
Number of Unique Values in Style: 8
Number of Unique Values in Country: 51


In [44]:
df_ramen['Country'].unique()

array(['Japan', 'China', 'Hong Kong', 'India', 'Thailand',
       'United States', 'South Korea', 'Malaysia', 'Indonesia', 'Taiwan',
       'Vietnam', 'Peru', 'Brazil', 'Singapore', 'Russian Federation',
       'UK', 'Spain', 'France', 'Souh Korea', 'Israel', 'Mexico',
       'Canada', 'Portugal', 'United Kingdom', 'Italy', 'Russia',
       'Philippines', 'Bangladesh', 'Australia', 'New Zealand', 'Ukraine',
       'Sarawak', 'Netherlands', 'Poland', 'Germany', 'USA', 'Phlippines',
       'Nigeria', 'Ghana', 'Hungary', 'Fiji', 'Pakistan', 'Nepal',
       'Myanmar', 'Cambodia', 'Finland', 'Sweden', 'Colombia', 'Estonia',
       'Holland', 'Dubai'], dtype=object)

In [45]:
brand_country = df_ramen['Brand'] + ' ' + df_ramen['Country']
brand_country.nunique()

#Number of Unique Values in Brand: 543 is less than 623
#possible for one brand to have multiple manufacturers in different countries

623

In [46]:
unique_brand = df_ramen['Brand'].unique().tolist()
sorted(unique_brand)[:20]

['1 To 3 Noodles',
 '7 Select',
 '7 Select/Nissin',
 '7-Eleven / Nissin',
 'A-One',
 'A-Sha',
 'A-Sha Dry Noodle',
 'A1',
 'ABC',
 'ABC President',
 'Acecook',
 'Adabi',
 'Ah Lai',
 'Aji-no-men',
 'Ajinatori',
 'Ajinomoto',
 'Alhami',
 'Amianda',
 'Amino',
 "Annie Chun's"]

##### Suspicious brand names in the beginning can be seen

#### Pick four brand names and find their similar names in the Brand column. Since we're matching the Brand column with itself, the result would always include the selected name with a score of 100.

### Token Sort Ratio
####  Sorts the strings alphabetically and joins them together, then calculate the distance ratio

In [47]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_sort_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 70),
 ('Jinbo Selection', 61),
 ('Seven & i', 53),
 ('Seven & I', 53)]

##### '7 Select/Nissin' has 70% similarity when referring to '7 Select'.

In [48]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_sort_ratio)

[('A-Sha', 100), ('Shan', 67), ('Daisho', 55), ('Nasoya', 55), ('Alhami', 55)]

##### Nothing below 70% to match A-Sha

In [49]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_sort_ratio)

[('Acecook', 100),
 ('Vina Acecook', 74),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

In [50]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_sort_ratio)

[("Chef Nic's Noodles", 100),
 ("Mr. Lee's Noodles", 71),
 ('Fantastic Noodles', 69),
 ('1 To 3 Noodles', 62),
 ("Mom's Dry Noodle", 59)]

##### Token sort ratio scorer will gets the wrong pair of Chef Nic's Noodles - Mr. Lee's Noodles at 70% threshold

In [51]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_sort_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

### Token Set Ratio
#### Similar to Sort Ratio except it takes out the common tokens before calculating the ratio between the new strings

In [52]:
process.extract('7 Select', unique_brand, scorer=fuzz.token_set_ratio)

[('7 Select', 100),
 ('7 Select/Nissin', 100),
 ('The Ramen Rater Select', 86),
 ('Jinbo Selection', 61),
 ('Seven & i', 53)]

##### Since the token set ratio is more flexible, the score has increased from 70% to 100% for 7 Select - 7 Select/Nissin.

In [53]:
process.extract('A-Sha', unique_brand, scorer=fuzz.token_set_ratio)

[('A-Sha', 100),
 ('A-Sha Dry Noodle', 100),
 ('Shan', 67),
 ('Ao Ba Ba', 60),
 ('Daisho', 55)]

##### A-Sha has another name as A-Sha Dry Noodle. And we can see this only by using token set ratio.

In [54]:
process.extract('Acecook', unique_brand, scorer=fuzz.token_set_ratio)

[('Acecook', 100),
 ('Vina Acecook', 100),
 ('Yatekomo', 53),
 ('Sahmyook', 53),
 ('Panco', 50)]

In [55]:
process.extract("Chef Nic's Noodles", unique_brand, scorer=fuzz.token_set_ratio)

[("Chef Nic's Noodles", 100),
 ('S&S', 100),
 ('Mr. Noodles', 82),
 ("Mr. Lee's Noodles", 72),
 ('Tseng Noodles', 70)]

In [56]:
process.extract('Chorip Dong', unique_brand, scorer=fuzz.token_set_ratio)

[('Chorip Dong', 100),
 ('ChoripDong', 95),
 ('Hi-Myon', 56),
 ('Mr. Udon', 56),
 ('Maison de Coree', 54)]

## Apply FuzzyWuzzy in one column

### Token Sort Ratio

In [57]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,)+i
             for x in unique_brand
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]

In [58]:
#Create dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort

,brand_sort,match_sort,score_sort
0,Higashimaru,Higashimaru,100
1,Higashimaru,Ishimaru,84
2,Higashimaru,Higashi,78
3,Higashimaru,Igarashi Seimen,54
4,Higashimaru,Nongshim,53
5,Single Grain,Single Grain,100
6,Single Grain,Pringles,60
7,Single Grain,Igarashi Seimen,59
8,Single Grain,Lien Ying,57
9,Single Grain,Vite Ramen,55


##### Since we're looking for matched values from the same column, one value pair would have another same pair in a reversed order. To eliminate one of them later, we need to find “representative” values for the same pairs.

##### Based on the tests above, better to just prioritize those that are at least 80% similarity. Also those which match to themselves (brand value and match value are exactly the same), and those which are duplicated pairs.

In [60]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) & 
                                  (similarity_sort['brand_sort'] != similarity_sort['match_sort']) &
                                  (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'] )]

In [61]:
high_score_sort.head()

,brand_sort,match_sort,score_sort,sorted_brand_sort
1,Higashimaru,Ishimaru,84,Higashimaru
56,MAMA,Mama,100,MAMA
81,Tasty ly,Tasty.ly,100,Tasty ly
226,ABC President,President,82,ABC President
232,Mamee Shinsegae,Mamee-Shinsegae,100,Mamee Shinsegae


In [62]:
#Drop the representative value column
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

In [63]:
#Group matches by brand names and scores
#pd.set_option('display.max_rows', None)
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

,,match_sort
brand_sort,score_sort,
Wu Mu,100,Wu-Mu
Lau Liu Tou,100,Lau Liu tou
Seven & I,100,Seven & i
TableMark,100,Tablemark
Nissin Miojo,100,Nissin-Miojo
Tasty ly,100,Tasty.ly
MyKuali,100,Mykuali
Mamee / Shinsegae,100,"Mamee Shinsegae, Mamee-Shinsegae"
MAMA,100,Mama


##### From the score of 91 and above, everything looks good. In each pair, the two values might have typos, one missing/extra character, or inconsistent format, but overall they obviously refer to each other. Below 91, it would be harder to tell. 

In [64]:
df_ramen[(df_ramen['Brand'] == 'Souper') | (df_ramen['Brand'] == 'Super')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars,T
2885,817,Souper,Bowl Ramen Shrimp,Bowl,Taiwan,3.75,NaN
3289,413,Souper,Alimentary Paste Chow Mein,Pack,Taiwan,4.25,NaN
825,2877,Super,Cup Instant Noodle Seafood Flavour,Cup,Malaysia,2.5,NaN
894,2808,Super,Super Cup Signature White Curry,Cup,Malaysia,0.25,NaN
1912,1790,Super,Kimchi,Cup,Malaysia,4.5,NaN
1914,1788,Super,Curry Flavour Noodles,Cup,Malaysia,3,NaN
1917,1785,Super,Instant Noodles Black Pepper Crab Mi Goreng,Cup,Malaysia,3.5,NaN
1970,1732,Super,Seafood Flavour Noodles,Cup,Malaysia,3.25,NaN
2207,1495,Super,Instant Noodles Mee Goreng,Cup,Malaysia,3.75,NaN


##### For this pair, we see that these two brands come from different manufacturers, and there's also no similarity in their ramen types or styles.These brands are not the same.

In [65]:
df_ramen[(df_ramen['Brand'] == 'Sura') | (df_ramen['Brand'] == 'Suraj')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars,T
3058,644,Sura,Knife Cut Rice Noodle,Bowl,South Korea,3.75,NaN
587,3115,Suraj,Masala Noodles,Pack,Canada,2,NaN


##### Two different manufacturer, two different brands

In [66]:
df_ramen[(df_ramen['Brand'] == 'Ped Chef') | (df_ramen['Brand'] == 'Red Chef')].sort_values(['Brand'])

,Review #,Brand,Variety,Style,Country,Stars,T
939,2763,Ped Chef,Sesame Clear Soup Noodles,Pack,Malaysia,3.5,NaN
248,3454,Red Chef,Spicy Sakura Prawn Soup Rice Vermicelli & Noodles (New Recipe),Pack,Malaysia,5,NaN
265,3437,Red Chef,Green Tom Yum Soup Noodles (New Formulation Dec 2019),Pack,Malaysia,5,NaN
466,3236,Red Chef,Spicy Sakura Prawn Soup Rice Vermicelli & Noodles,Pack,Malaysia,5,NaN
500,3202,Red Chef,Pandan White Curry Noodle (New Recipe),Pack,Malaysia,5,NaN
940,2762,Red Chef,Pandan White Curry Noodles,Pack,Malaysia,5,NaN
941,2761,Red Chef,Green Tom Yum Soup Noodles,Pack,Malaysia,5,NaN
942,2760,Red Chef,Spicy Sakura Prawn Soup Noodles,Pack,Malaysia,5,NaN


##### These two brands are the same

### Token Set Ratio
##### Similar steps to Token Sort

In [67]:
#Create tuples of brand names, matched brand names, and the score
score_set = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_set_ratio)]

In [76]:
#Create dataframe from the tuples
similarity_set = pd.DataFrame(score_set, columns=['brand_set','match_set','score_set'])
similarity_set.head()

,brand_set,match_set,score_set
0,Higashimaru,Higashimaru,100
1,Higashimaru,Ishimaru,84
2,Higashimaru,Higashi,78
3,Higashimaru,Igarashi Seimen,54
4,Higashimaru,Nongshim,53


In [69]:
#Derive representative values
similarity_set['sorted_brand_set'] = np.minimum(similarity_set['brand_set'], similarity_set['match_set'])

#Pick values
high_score_set = similarity_set[(similarity_set['score_set'] >= 80) & 
                                    (similarity_set['brand_set'] != similarity_set['match_set']) & 
                                    (similarity_set['sorted_brand_set'] != similarity_set['match_set'])]

#Drop the representative value column
high_score_set = high_score_set.drop('sorted_brand_set',axis=1).copy()

##### Since the token set ratio scorer will tolerate more 'noise' when matching two values, I thus can group the result by matched values to reduce the number of rows.

In [70]:
#Group brands by matches and scores
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
high_score_set.groupby(['match_set','score_set']).agg(
                       {'brand_set': ', '.join}).sort_values(
                       ['score_set'], ascending=False)

,,brand_set
match_set,score_set,
7 Select/Nissin,100,7 Select
S&S,100,"Mr. Lee's Noodles, Mom's Dry Noodle, Chef Nic's Noodles, Mama Pat's, Culley's, Mike's Mighty Good Craft Ramen, Ching's Secret, Dr. McDougall's, Farmer's Heart, Chaudhary's Wai Wai, Campbell's, Annie Chun's, Han's South Korea, Kim's Bowl"
Sichuan Baijia,100,Baijia
Seven & i,100,Seven & I
Samyang Foods,100,Samyang
Samurai Ramen,100,Samurai
Sakurai Foods,100,Sakurai
Wei Wei,100,"Wei Lih, Chuan Wei Wang, Wei Chuan"
Q,100,Little Couples Q Noodle


### Comparison

Create a merged table including results from token sort ratio and token set ratio with some changes.
The tables will be merged by matched values to shorten the result table, and in order to keep the scores after grouping all values, create new columns which combine brand names and scores.

In [71]:
#Create columns with brand names combining scores
high_score_sort['brand_sort'] = high_score_sort['brand_sort'] + ': ' + high_score_sort['score_sort'].astype(str)
high_score_set['brand_set'] = high_score_set['brand_set'] + ': ' + high_score_set['score_set'].astype(str)

In [72]:
#Group data by matched name and store in new dataframe
token_sort = high_score_sort.groupby(['match_sort']).agg({'brand_sort': ', '.join}).reset_index()
token_set = high_score_set.groupby(['match_set']).agg({'brand_set': ', '.join}).reset_index()

#Rename columns
token_sort = token_sort.rename(columns={'match_sort':'brand'})
token_set = token_set.rename(columns={'match_set':'brand'})

In [73]:
#Outer join two tables by brand (matched names)
similarity = pd.merge(token_sort, token_set, how='outer', on='brand')

#Replace NaN values and rename columns for readability
similarity = similarity.replace(np.nan,'')
similarity = similarity.rename(columns={'brand_set':'token_set_ratio','brand_sort':'token_sort_ratio'})

In [74]:
similarity.sort_values('brand').reset_index(drop=True)

,brand,token_sort_ratio,token_set_ratio
0,7 Select/Nissin,,7 Select: 100
1,7-Eleven / Nissin,,7 Select/Nissin: 80
2,A-Sha Dry Noodle,,A-Sha: 100
3,ABC President,,ABC: 100
4,Baixiang Noodles,,Baixiang: 100
5,ChoripDong,Chorip Dong: 95,Chorip Dong: 95
6,Fantastic Noodles,,Fantastic: 100
7,Fashion Foods,Fashion Food: 96,Fashion Food: 96
8,GB Company,,Fu Chang Chinese Noodle Company: 82
9,Goku-Uma,Goku Uma: 100,Goku Uma: 100


##### The two methods have differences.Token set ratio matches wrong names with high score, however, it does bring in more matches compared to the token sort ratio. This means to get the most matches, one should use both of the scorers.